In [113]:
import pandas as pd
import os
import glob
import yfinance as yf
from datetime import datetime
import math

In [ ]:
"""## PARA DESCARGAR LA NUEVA INFORMACION - TARDÓ 120 Minutos, 87 Minutos 
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Descargar los datos de Yahoo Finance y almacenarlos en un DataFrame por cada stock
for archivo_csv in archivos_csv:
    # Obtener el símbolo del stock desde el nombre del archivo
    nombre_archivo = os.path.basename(archivo_csv)
    simbolo = nombre_archivo.split('.')[0]  # Suponemos que el nombre del archivo es el símbolo del stock
    
    # Descargar los datos desde Yahoo Finance para los últimos 10 años
    data = yf.download(simbolo, start='2013-01-01')
    
    # Filtrar las columnas necesarias (Precio de apertura, cierre, cierre ajustado y volumen)
    data = data[['Open', 'Close', 'Adj Close', 'Volume']]
    
    # Guardar los datos en un nuevo archivo CSV
    nuevo_nombre_archivo = f'{simbolo}.csv'
    ruta_nuevo_archivo = os.path.join(directorio, nuevo_nombre_archivo)
    data.to_csv(ruta_nuevo_archivo)
    
    print(f"Se descargaron los datos para el stock {simbolo} y se guardaron en {nuevo_nombre_archivo}.") """

In [2]:
## PARA EXTRAER LOS NOMBRES DE LOS DOCUMENTOS 
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Lista para almacenar los nombres de los archivos CSV
nombres_archivos_csv = []

# Extraer el nombre de cada archivo CSV
for archivo in archivos_csv:
    nombres_archivos_csv.append(os.path.basename(archivo))

In [3]:
## PARA CREAR LOS DATAFRAME
# Lista para almacenar los DataFrames de los archivos CSV
dataframes_csv = []
# Leer cada archivo CSV y almacenar el DataFrame correspondiente
for i in range(len(nombres_archivos_csv)):
    ruta_archivo = directorio + '/' + nombres_archivos_csv[i]  # Ruta completa al archivo
    df = pd.read_csv(ruta_archivo)
    dataframes_csv.append(df)

# Crea el DF por accion
for i in range(len(nombres_archivos_csv)):
    # Extraer un nombre de la lista
    nombre_variable = nombres_archivos_csv[i].replace('.csv','')
    # Asignar un valor a la variable con el nombre extraído
    globals()[nombre_variable] = dataframes_csv[i]

In [4]:
# Extraer sólo el nombre del stock
for i in range(len(nombres_archivos_csv)):
    nombres_archivos_csv[i] = nombres_archivos_csv[i].replace('.csv','')

In [5]:
## PARA ELIMINAR LOS STOCKS INÚTILES
archivos_inutiles = []
length = len(globals()[nombres_archivos_csv[0]]['Date'])
ult_fecha = A['Date'][len(A['Date'])-1]

# Iterar sobre los nombres de archivos CSV
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    fecha = dataframe['Date']
    
    # Se eliminan los df que no tengan datos, que no tengan histórico desde el primer día hábil del 2013,
    # que no estén cotizando para el día hábil más cercano de cuando descargamos los datos,
    # y que tengan información continua (que no falte ninguna cotización)
    if len(fecha) != length or fecha[0] != '2013-01-02'  or fecha[len(fecha) - 1] != ult_fecha:
        archivos_inutiles.append(nombre_archivo)

# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for nombre_archivo in archivos_inutiles:
    nombre_dataframe = nombre_archivo.replace('.csv', '')
    del globals()[nombre_dataframe]
    nombres_archivos_csv.remove(nombre_archivo)

In [97]:
## CALCULAR DESVIACIÓN ESTANDARD
import numpy as np
metrics = pd.DataFrame()

metrics['Stock'] = nombres_archivos_csv
std = []

for i in range(len(metrics['Stock'])):
    std.append(np.std(globals()[metrics['Stock'][i]]['Adj Close']))

metrics['Standard Deviation'] = std


In [98]:
## RANGO DE PRECIOS
r_min = []
r_max = []
fecha = '2023-07-14'    ## Fecha de 'Ayer', del dia anterior para basarse ena estimación del precio de "hoy" (Siempre tiene que ser de días hábiles)
indice = globals()[metrics['Stock'][0]].loc[A['Date'] == fecha].index[0]    ## Como todos los archivos tienen la misma cantidad de datos, si en uno se encuentra la fecha en x 
                                                                            ## posicion entonces en esa misma estará para todos los demás df
for i in range(len(metrics['Stock'])):
    r_min.append(globals()[metrics['Stock'][i]]['Close'][indice] - metrics['Standard Deviation'][i])
    r_max.append(globals()[metrics['Stock'][i]]['Close'][indice] + metrics['Standard Deviation'][i])

metrics['Rango Mínimo Estimado'] = r_min
metrics['Rango Máximo Estimado'] = r_max

In [99]:
## PARA CALCULAR LOS RENDIMIENTOS DIARIOS -> (Pf/Pi)-1
for j in range(len(nombres_archivos_csv)):
    nombre_archivo = nombres_archivos_csv[j]
    dataframe = globals()[nombre_archivo]
    dataframe['Rendimientos diarios (%)'] = np.nan
    adj_close = dataframe['Adj Close']
    rendimientos = [((adj_close[i] / adj_close[i-1]) - 1)*100 for i in range(1, len(adj_close))]
    dataframe['Rendimientos diarios (%)'][1:] = rendimientos    

C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_11676\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos diarios (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_11676\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos diarios (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_11676\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [100]:
## CALCULAR PROMEDIO DE RENDIMIENTOS DIARIOS POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(globals()[metrics['Stock'][i]]['Rendimientos diarios (%)']))
metrics['Rendimiento promedio diario (%)'] = p_rend

In [101]:
## CALCULAR PROMEDIO DE RENDIMIENTOS MENSUALES POR STOCK

# Crear un diccionario para almacenar los DataFrames con los rendimientos mensuales
rendimientos_mensuales = {}

# Calcular el rendimiento mensual para cada DataFrame
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    
    # Agrupar los datos por año y mes y obtener el primer y último valor de "Adj Close" de cada mes
    monthly_data = dataframe.resample('M', on='Date').agg({'Adj Close': ['first', 'last']})
    
    # Calcular el rendimiento del mes
    monthly_data['Rendimiento mensual (%)'] = (monthly_data['Adj Close']['last'] / monthly_data['Adj Close']['first'] - 1) * 100
    
    # Almacenar el DataFrame con los rendimientos mensuales en el diccionario
    rendimientos_mensuales[nombre_archivo] = monthly_data

# Ahora puedes acceder a los DataFrames individuales con sus nombres
# por ejemplo, dataframes_rendimientos_mensuales['NombreDelDataFrame']


In [102]:
## CALCULAR PROMEDIO DE RENDIMIENTOS MENSUALES POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(rendimientos_mensuales[metrics['Stock'][i]]['Rendimiento mensual (%)']))
metrics['Rendimiento promedio mensual (%)'] = p_rend

In [103]:
## CALCULAR PROMEDIO DE RENDIMIENTOS ANUALES POR STOCK

# Crear un diccionario para almacenar los DataFrames con los rendimientos mensuales
rendimientos_anuales = {}

# Calcular el rendimiento mensual para cada DataFrame
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    
    # Agrupar los datos por año y mes y obtener el primer y último valor de "Adj Close" de cada mes
    annual_data = dataframe.resample('A', on='Date').agg({'Adj Close': ['first', 'last']})
    
    # Calcular el rendimiento del mes
    annual_data['Rendimiento Anual (%)'] = (annual_data['Adj Close']['last'] / annual_data['Adj Close']['first'] - 1) * 100
    
    # Almacenar el DataFrame con los rendimientos mensuales en el diccionario
    rendimientos_anuales[nombre_archivo] = annual_data

# Ahora puedes acceder a los DataFrames individuales con sus nombres
# por ejemplo, dataframes_rendimientos_mensuales['NombreDelDataFrame']

In [104]:
## CALCULAR PROMEDIO DE RENDIMIENTOS ANUALES POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(rendimientos_anuales[metrics['Stock'][i]]['Rendimiento Anual (%)']))
metrics['Rendimiento promedio anual (%)'] = p_rend

In [117]:
## CALCULAR VOLATILIDAD METODO "Historical Volatility (HV)" 
    # HV = √VAR * √N
    # VAR = SD/N

var = []
HV = []
dias_negociados = len(A['Date'])
for i in range(len(metrics['Stock'])):
    var.append((metrics['Standard Deviation'][i])/(dias_negociados))
    HV.append(math.sqrt(var[i]) * math.sqrt(dias_negociados))



In [ ]:
## CALCULAR VOLATILIDAD METODO "Realized Volatility (RV)" 
    # RV = √((∑SDR)/N)
    # SDR = (Daily return)^2
SDR = []  # Para agregar la suma de cada stock

for j in range(len(metrics['Stock'])):
    sdr = []  # Reiniciar la lista de SDR para cada stock
    for i in range(1, dias_negociados):
        sdr.append((globals()[metrics['Stock'][j]]['Rendimientos diarios (%)'][i]) ** 2)
    SDR.append(sum(sdr))

# Ahora calculamos la volatilidad (RV) para cada stock
RV = [((sdr_sum / dias_negociados) ** 0.5) for sdr_sum in SDR]


In [149]:
sorted(HV,reverse=True)

[253425.94321099526,
 10137.612717160087,
 1371.8977890591689,
 886.9546460131476,
 292.0462452276294,
 184.0782210787176,
 174.16720559062648,
 164.82315219249892,
 109.93288562246697,
 85.89959862444735,
 76.85196117907611,
 68.19227358216476,
 59.19559788575734,
 55.67858032328726,
 53.98728212265971,
 41.877761645943174,
 41.53671648270268,
 40.19206338869625,
 39.06094147000429,
 38.51469498432499,
 35.06135123053116,
 31.29073441898565,
 29.989218224941453,
 27.61062849923603,
 25.532630585599854,
 24.635148468888534,
 24.124909862846522,
 24.11285715710587,
 23.89899331097209,
 22.477882794037185,
 22.451862562600382,
 22.305841736488347,
 22.131361834721915,
 20.745197563008272,
 20.70381269522667,
 19.533238532014742,
 18.76864756596832,
 18.702257809016988,
 17.22076342553248,
 17.021646459892764,
 16.335985745672946,
 15.830000811839447,
 15.694314653336011,
 15.67541051753544,
 15.479880472651821,
 15.45056781298608,
 15.44228787365043,
 15.044271078442927,
 15.001295590402

In [148]:
sorted(RV,reverse=True)

[257.4829587279806,
 94.78185246218767,
 55.04687646233772,
 47.443145038375846,
 44.62322087154467,
 40.12204284196556,
 27.28144845522766,
 25.529102385855875,
 17.42709503313854,
 17.068555195355245,
 15.421896437392563,
 14.909032615441106,
 13.808481986182693,
 13.45042584537864,
 11.814316403402058,
 11.51481322410925,
 11.467667763473894,
 11.249415322117036,
 11.229469430429411,
 10.804076353382824,
 10.644557064028543,
 10.577470742892976,
 10.380831172963996,
 10.178950412319445,
 10.129096453168657,
 10.070444453455242,
 9.994568385139893,
 9.922967298723028,
 9.875927926775304,
 9.813711504190492,
 9.802045049595423,
 9.642452399958986,
 9.423985050257008,
 9.365781489290173,
 9.314717994869861,
 9.23618254097145,
 9.112533407048053,
 9.112248247006498,
 9.043026440531095,
 9.009935468708125,
 8.916911291572678,
 8.818763559399413,
 8.798363270353345,
 8.797250688049289,
 8.706424160006275,
 8.528409559824489,
 8.389455354078201,
 8.308226176401186,
 8.286423990590775,
 8.2